# Skeleton-based action recognition using TAO PoseClassificationNet

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

<img align="center" src="https://developer.nvidia.com/sites/default/files/akamai/embedded-transfer-learning-toolkit-software-stack-1200x670px.png" width="1080">


## Learning Objectives

In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Train a model for skeleton-based action recognition on the [Kinetics](https://deepmind.com/research/open-source/kinetics) dataset.
* Evaluate the trained model.
* Run Inference on the trained model.
* Export the trained model to a .etlt file (encrypted ONNX model) for deployment to DeepStream or TensorRT.
* Convert the pose data from [deepstream-bodypose-3d](https://github.com/NVIDIA-AI-IOT/deepstream_reference_apps/tree/master/deepstream-bodypose-3d) to skeleton arrays for inference.

## Table of Contents

This notebook shows an example usecase of PoseClassificationNet using Train Adapt Optimize (TAO) Toolkit.

1. [Set up env variables and map drives](#head-1)
2. [Prepare dataset and pre-trained model](#head-2)
3. [Provide training specification](#head-3)
4. [Run TAO training](#head-4)
5. [Evaluate trained models](#head-5)
6. [Inferences](#head-6)
8. [Convert pose data](#head-8)


## Connect to a GPU Runtime

1.   Change Runtime type to GPU by Runtime(Top Left tab)->Change Runtime Type->GPU(Hardware Accelerator)
2.   Then click on Connect (Top Right)



## Mounting Google drive
Mount your Google drive storage to this Colab instance

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##Setup Python Environment
Setup the environment necessary to run the TAO Networks by running the bash script

In [ ]:
!sh /content/drive/MyDrive/ColabNotebooks/pytorch/setup_env.sh

## 1. Set up env variables and map drives <a class="anchor" id="head-1"></a>

When using the purpose-built pretrained models from NGC, please make sure to set the `$KEY` environment variable to the key as mentioned in the model overview. Failing to do so, can lead to errors when trying to load them as pretrained models.

The TAO launcher uses docker containers under the hood, and **for our data and results directory to be visible to the docker, they need to be mapped**. The launcher can be configured using the config file `~/.tao_mounts.json`. Apart from the mounts, you can also configure additional options like the Environment Variables and amount of Shared Memory available to the TAO launcher. <br>

`IMPORTANT NOTE:` The code below creates a sample `~/.tao_mounts.json`  file. Here, we can map directories in which we save the data, specs, results and cache. You should configure it for your specific case so these directories are correctly visible to the docker container.


In [ ]:
%env DATA_DIR=/content/data
# note: You could set the SPECS_DIR to folder of the experiments specs downloaded with the notebook
%env LOCAL_PROJECT_DIR=/content
%env SPECS_DIR=/content/drive/MyDrive/ColabNotebooks/pytorch/cv_notebooks/pose_classification_net/specs
%env RESULTS_DIR=/content/results

# Set your encryption key, and use the same key for all commands
%env KEY = nvidia_tao

In [ ]:
! mkdir -p $DATA_DIR
! mkdir -p $SPECS_DIR
! mkdir -p $RESULTS_DIR

## 2. Prepare dataset and pre-trained model <a class="anchor" id="head-2"></a>
We will be using the NVIDIA dataset generated by [deepstream-bodypose-3d](https://github.com/NVIDIA-AI-IOT/deepstream_reference_apps/tree/master/deepstream-bodypose-3d). 

Download the NVIDIA dataset and extract the files.

In [ ]:
# Download the dataset
!pip3 install gdown
!gdown https://drive.google.com/uc?id=1GhSt53-7MlFfauEZ2YkuzOaZVNIGo_c- -O $DATA_DIR/data_3dbp_nvidia.zip

In [ ]:
# Extract the files
!mkdir -p $DATA_DIR/nvidia
!unzip $DATA_DIR/data_3dbp_nvidia.zip -d $DATA_DIR/nvidia
!rm $DATA_DIR/data_3dbp_nvidia.zip

In [ ]:
# Verify
!ls -l $DATA_DIR/nvidia

 We also provide scripts to process the [Kinetics](https://deepmind.com/research/open-source/kinetics) dataset for the tutorial. Download the pre-processed data of Kinetics-Skeleton [here](https://drive.google.com/uc?id=1dmzCRQsFXJ18BlXj1G9sbDnsclXIdDdR) and extract them first:
 The following cells for processing the Kinetics dataset is `Optional`.
 Kinetics is the bigger dataset when compared to the NVIDIA dataset

In [ ]:
# # download the dataset.
# !pip3 install gdown
# !gdown https://drive.google.com/uc?id=1dmzCRQsFXJ18BlXj1G9sbDnsclXIdDdR -O $DATA_DIR/st-gcn-processed-data.zip

In [ ]:
# # extract the files
# !unzip $DATA_DIR/st-gcn-processed-data.zip -d $DATA_DIR
# !mv $DATA_DIR/data/Kinetics/kinetics-skeleton $DATA_DIR/kinetics
# !rm -r $DATA_DIR/data
# !rm $DATA_DIR/st-gcn-processed-data.zip

In [ ]:
# # verify
# !ls -l $DATA_DIR/kinetics

In [ ]:
# # select actions
# import os
# import pickle
# import numpy as np

# data_dir = os.path.join(os.environ["DATA_DIR"], "kinetics")

# # front_raises: 134
# # pull_ups: 255
# # clean_and_jerk: 59
# # presenting_weather_forecast: 254
# # deadlifting: 88
# selected_actions = {
#     134: 0,
#     255: 1,
#     59: 2,
#     254: 3,
#     88: 4
# }

# def select_actions(selected_actions, data_dir, split_name):
#     """Select a subset of actions and their corresponding labels.
    
#     Args:
#         selected_actions (dict): Map from selected class IDs to new class IDs.
#         data_dir (str): Path to the directory of data arrays (.npy) and labels (.pkl).
#         split_name (str): Name of the split to be processed, e.g., "train" and "val".
        
#     Returns:
#         No explicit returns
#     """
#     data_path = os.path.join(data_dir, f"{split_name}_data.npy")
#     label_path = os.path.join(data_dir, f"{split_name}_label.pkl")

#     data_array = np.load(file=data_path)
#     with open(label_path, "rb") as label_file:
#         labels = pickle.load(label_file)

#     assert(len(labels) == 2)
#     assert(data_array.shape[0] == len(labels[0]))
#     assert(len(labels[0]) == len(labels[1]))

#     print(f"No. total samples for {split_name}: {data_array.shape[0]}")

#     selected_indices = []
#     for i in range(data_array.shape[0]):
#         if labels[1][i] in selected_actions.keys():
#             selected_indices.append(i)

#     data_array = data_array[selected_indices, :, :, :, :]
#     selected_sample_names = [labels[0][x] for x in selected_indices]
#     selected_labels = [selected_actions[labels[1][x]] for x in selected_indices]
#     labels = (selected_sample_names, selected_labels)

#     print(f"No. selected samples for {split_name}: {data_array.shape[0]}")

#     np.save(file=data_path, arr=data_array, allow_pickle=False)
#     with open(label_path, "wb") as label_file:
#         pickle.dump(labels, label_file, protocol=4)

# select_actions(selected_actions, data_dir, "train")
# select_actions(selected_actions, data_dir, "val")

`OPTIONAL:` Download the pretrained model from NGC. We will use NGC CLI to get the data and model. For more details, go to https://ngc.nvidia.com and click the SETUP on the navigation bar.

In [ ]:
# Installing NGC CLI on the local machine.
## Download and install
import os
%env CLI=ngccli_cat_linux.zip
!mkdir -p $LOCAL_PROJECT_DIR/ngccli

# Remove any previously existing CLI installations
!rm -rf $LOCAL_PROJECT_DIR/ngccli/*
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $LOCAL_PROJECT_DIR/ngccli
!unzip -u "$LOCAL_PROJECT_DIR/ngccli/$CLI" -d $LOCAL_PROJECT_DIR/ngccli/
!rm $LOCAL_PROJECT_DIR/ngccli/*.zip 
os.environ["PATH"]="{}/ngccli/ngc-cli:{}".format(os.getenv("LOCAL_PROJECT_DIR", ""), os.getenv("PATH", ""))
!cp /usr/lib/x86_64-linux-gnu/libstdc++.so.6 /content/ngccli/ngc-cli/libstdc++.so.6

In [ ]:
!ngc registry model list nvidia/tao/poseclassificationnet:*

In [ ]:
!mkdir -p $RESULTS_DIR/pretrained

In [ ]:
# Pull pretrained model from NGC 
!ngc registry model download-version "nvidia/tao/poseclassificationnet:trainable_v1.0" --dest $RESULTS_DIR/pretrained

In [ ]:
print("Check that model is downloaded into dir.")
!ls -l $RESULTS_DIR/pretrained/poseclassificationnet_vtrainable_v1.0

## 3. Provide training specification <a class="anchor" id="head-3"></a>

We provide specification files to configure the training parameters including:

* model_config: configure the model setting
    * model_type: type of model, ST-GCN
    * in_channels: number of input channels
    * num_class: number of classes
    * dropout: probability to drop the hidden units
    * graph_layout: type of graph layout, nvidia/openpose/human3.6m/ntu-rgb+d/ntu_edge/coco
    * graph_strategy: type of graph strategy, uniform/distance/spatial
    * edge_importance_weighting: enabling edge importance weighting
* train_config: configure the training hyperparameters
    * optim
    * epochs
    * checkpoint_interval
    * grad_clip
* dataset_config: configure the dataset and augmentation methods
    * train_data_path
    * train_label_path
    * val_data_path
    * val_label_path
    * label_map
    * random_choose
    * random_move
    * window_size
    * batch_size
    * workers: number of workers to do data loading

Please refer to the TAO documentation about PoseClassificationNet to get all the parameters that are configurable.

In [ ]:
!cat $SPECS_DIR/train_nvidia.yaml

## 4. Run TAO training <a class="anchor" id="head-4"></a>
* Provide the sample spec file and the output directory location for models.
* WARNING: Training will take several hours or one day to complete.

### 4.1 Train NVIDIA model

In [ ]:
print("Train model from scratch")
!tao pose_classification train \
                  -e $SPECS_DIR/train_nvidia.yaml \
                  -r $RESULTS_DIR/nvidia \
                  -k $KEY

We provide pre-trained ST-GCN model trained on the NVIDIA dataset. With the pre-trained model, we can even get better accuracy with less epochs.

In [ ]:
# print("To resume training from a checkpoint, set the resume_training_checkpoint_path option to be the .tlt you want to resume from")
# print("remember to remove the `=` in the checkpoint's file name")
# !tao pose_classification train \
#                   -e $SPECS_DIR/train_nvidia.yaml \
#                   -r $RESULTS_DIR/nvidia \
#                   -k $KEY \
#                   resume_training_checkpoint_path=

In [ ]:
print('Encrypted checkpoints:')
print('---------------------')
!ls -ltrh $RESULTS_DIR/nvidia

In [ ]:
print('Rename a model:')
print('---------------------')
# NOTE: The following command may require `sudo`. You can run the command outside the notebook.
!find $RESULTS_DIR/nvidia/ -name *epoch=69* | xargs realpath | xargs -I {} mv {} $RESULTS_DIR/nvidia/nvidia_model.tlt
!ls -ltrh $RESULTS_DIR/nvidia/nvidia_model.tlt

### `OPTIONAL` 4.2 Train Kinetics model

We will train a Kinetics model from scratch.

In [ ]:
# print("Train model")
# !tao pose_classification train \
#                   -e $SPECS_DIR/train_kinetics.yaml \
#                   -r $RESULTS_DIR/kinetics \
#                   -k $KEY

In [ ]:
# print('Encrypted checkpoints:')
# print('---------------------')
# !ls -ltrh $RESULTS_DIR/kinetics

In [ ]:
# print('Rename a model:')
# print('---------------------')
# # NOTE: The following command may require `sudo`. You can run the command outside the notebook.
# !find $RESULTS_DIR/kinetics/ -name *epoch=49* | xargs realpath | xargs -I {} mv {} $RESULTS_DIR/kinetics/kinetics_model.tlt
# !ls -ltrh $RESULTS_DIR/kinetics/kinetics_model.tlt

## 5. Evaluate trained models <a class="anchor" id="head-5"></a>
Evaluate trained model.

In [ ]:
!tao pose_classification evaluate \
                    -e $SPECS_DIR/evaluate_nvidia.yaml \
                    -k $KEY \
                    model=$RESULTS_DIR/nvidia/nvidia_model.tlt \
                    data=$DATA_DIR/nvidia/val_data.npy \
                    label=$DATA_DIR/nvidia/val_label.pkl

## 6. Inferences <a class="anchor" id="head-6"></a>
In this section, we run the pose classification inference tool to generate inferences with the trained models and save the results under `$RESULTS_DIR`. 

In [ ]:
!tao pose_classification inference \
                    -e $SPECS_DIR/infer_nvidia.yaml \
                    -k $KEY \
                    model=$RESULTS_DIR/nvidia/nvidia_model.tlt \
                    data=$DATA_DIR/nvidia/val_data.npy \
                    output_file=$RESULTS_DIR/results.txt

## `OPTIONAL` 8. Convert pose data <a class="anchor" id="head-8"></a>
Convert the JSON pose data from [deepstream-bodypose-3d](https://github.com/NVIDIA-AI-IOT/deepstream_reference_apps/tree/master/deepstream-bodypose-3d) to NumPy arrays for inference.

In [ ]:
# !mkdir -p $RESULTS_DIR/dataset_convert

In [ ]:
# !tao pose_classification dataset_convert \
#                    -e $SPECS_DIR/dataset_convert_nvidia.yaml \
#                    -k $KEY \
#                    data=/absolute/path/to/your/json/pose/data \
#                    output_dir=$RESULTS_DIR/dataset_convert

In [ ]:
# print('Converted pose data:')
# print('------------')
# !ls -lth $RESULTS_DIR/dataset_convert

This notebook has come to an end.